In [1]:
import pandas as pd
import numpy as np

In [2]:
rev1 = pd.read_csv("../reviews/1429_1.csv")
rev2 = pd.read_csv("../reviews/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")
rev3 = pd.read_csv("../reviews/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../reviews/1429_1.csv'

In [3]:
# print(rev1.columns, rev2.columns, rev3.columns)
# Find common columns across all three dataframes using set intersection
common_columns = list(set(rev1.columns) & set(rev2.columns) & set(rev3.columns))
print("Common columns across all three dataframes:")
print(sorted(common_columns))
print(f"\nNumber of common columns: {len(common_columns)}")


Common columns across all three dataframes:
['asins', 'brand', 'categories', 'id', 'keys', 'manufacturer', 'name', 'reviews.date', 'reviews.dateSeen', 'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title', 'reviews.username']

Number of common columns: 17


In [3]:
# Combine all three dataframes using only the common columns
combined_df = pd.concat([rev1[common_columns], rev2[common_columns], rev3[common_columns]], 
                       ignore_index=True)

print(f"Combined dataframe shape: {combined_df.shape}")
print(f"Combined dataframe columns: {list(combined_df.columns)}")
combined_df.head()


NameError: name 'rev1' is not defined

In [4]:
combined_df.columns.sort_values()

NameError: name 'combined_df' is not defined

In [5]:
review_df_sementic = combined_df[["reviews.title", "reviews.text", "reviews.rating"]]
review_df_sementic.sample(5)

NameError: name 'combined_df' is not defined

In [6]:
review_df_sementic.isnull().sum()
review_df_sementic.dropna(inplace=True)
review_df_sementic.duplicated().sum()
review_df_sementic.drop_duplicates(inplace=True)

NameError: name 'review_df_sementic' is not defined

In [7]:
review_df_sementic.loc[:, "rating_semantic"] = review_df_sementic["reviews.rating"].apply(lambda x: 1 if x > 3 else -1 if x < 3 else 0)


NameError: name 'review_df_sementic' is not defined

In [8]:
import torch
# Use the yangheng/deberta-v3-large-absa-v1.1 model with proper tokenizer configuration
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Load the model with trust_remote_code=True to handle custom tokenizer
model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, trust_remote_code=True)

# Create aspect-based sentiment pipeline
absa_pipe = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True,
    trust_remote_code=True
)

# Function to get sentiment scores using the ABSA model
def get_absa_sentiment(text):
    try:
        result = absa_pipe(text[:512])  # Truncate to avoid token limits
        # For ABSA model, we need to check the label mapping
        # Usually positive sentiment has higher score
        return result[0]
    except Exception as e:
        print(f"Error processing text: {e}")
        return [{"label": "neutral", "score": 0.5}]

# Function to convert ABSA result to binary rating
def absa_to_rating(absa_result, threshold=0.5):
    # Find the highest scoring label
    best_score = max(absa_result, key=lambda x: x['score'])
    return 1 if best_score['score'] > threshold else 0


e:\Hackathons\Amazon HackOn\Models\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\Hackathons\Amazon HackOn\Models\venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0
e:\Hackathons\Amazon HackOn\Models\venv\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality us

In [61]:
%%time
text = review_df_sementic['reviews.text'].sample().to_list()[0]
print(text, absa_pipe(text), get_absa_sentiment(text))


It's exactly what you would expect for a tablet under $100. Very basic. Screen resolution if sub par for what's out today. Speed is questionable but works for our 2 year old. [[{'label': 'Negative', 'score': 0.3285614252090454}, {'label': 'Neutral', 'score': 0.35901209712028503}, {'label': 'Positive', 'score': 0.31242650747299194}]] [{'label': 'Negative', 'score': 0.3285614252090454}, {'label': 'Neutral', 'score': 0.35901209712028503}, {'label': 'Positive', 'score': 0.31242650747299194}]
CPU times: total: 219 ms
Wall time: 211 ms


In [114]:
# Function to process a list of texts and return ratings
def get_rating_semantic_batch(text_list):
    ratings = []
    for text in text_list:
        try:
            absa_result = get_absa_sentiment(text)
            # Convert to -1, 0, 1 rating based on sentiment scores
            # Find the highest scoring label by comparing scores directly
            max_score = 0
            best_label = 'neutral'
            for item in absa_result:
                if abs(item['score']) > abs(max_score):
                    max_score = item['score']
                    best_label = item['label']
            
            
            if best_label == 'Positive':
                ratings.append(1)
            elif best_label == 'Negative':
                ratings.append(-1)
            else:
                ratings.append(0)
        except Exception as e:
            print(f"Error processing text: {e}")
            ratings.append(0)
    return ratings


In [115]:
sample = review_df_sementic.sample(1000)

np.int64(20661)

In [116]:
sample.loc[:, "rating_semantic_generated"] = get_rating_semantic_batch(sample["reviews.text"].tolist())

In [133]:
sample.loc[sample['rating_semantic_generated'] == 0]

,reviews.title,reviews.text,reviews.rating,rating_semantic,rating_semantic_generated
62581,nice tablet for the price!,I purchased this for my wife so that she has a...,5.0,1,0
39270,Good for the money,Batteries for my MP3 player it takes one I sle...,4.0,1,0
38276,Five Stars,What can I say battery's,5.0,1,0
2408,Outstanding speed,I bought this as a gift and the recipient love...,5.0,1,0
34706,2-year Review,I buy AAA and AA batteries about 4 - 6 times a...,5.0,1,0
64986,Tablet,"This is a first for an adult. Price is right, ...",5.0,1,0
28830,Very good tablet,My sister told me about the Kindle Fire and I ...,5.0,1,0
21747,"The 7"" Amazon Kindle Fire is the best starter ...","The 7"" Amazon Kindle Fire is the best starter ...",5.0,1,0
13308,Good for what I need it for,Wanted something to take with me. (I don't use...,4.0,1,0
36059,Five Stars,Batteries are batteries.,5.0,1,0


In [123]:
accuracy = (sample["rating_semantic_generated"] == sample["rating_semantic"]).mean()
accuracy

np.float64(0.883)

In [ ]:
combined_df.isnull().sum()
combined_df.drop(columns=["reviews.numHelpful", "reviews.id", "name", "reviews.doRecommend"], inplace=True)
combined_df.dropna()
combined_df.shape

(67992, 13)

In [172]:
combined_df.duplicated().sum()
combined_df.drop_duplicates(inplace=True)

In [173]:
combined_df.to_csv("../reviews/combined_reviews.csv")

# Aspect generation for each product

In [168]:
top_5_reviewed = combined_df["id"].value_counts()[:5].index
top_5_reviewed

Index(['AVphgVaX1cnluZ0-DR74', 'AVpgNzjwLJeJML43Kpxn', 'AVpfl8cLLJeJML43AE3S',
       'AV1YE_muvKc47QAVgpwE', 'AVpe7xlELJeJML43ypLz'],
      dtype='object', name='id')

In [11]:
import pandas as pd
from pyabsa import AspectTermExtraction as ATEPC
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
import os

# STEP 1: Get product ID from user
product_id = "AVphgVaX1cnluZ0-DR74"

# STEP 2: Load reviews using pandas
# df = pd.read_csv("..\\reviews\\combined_reviews.csv")  # replace with your CSV path
# product_reviews = df[df["id"] == product_id]["reviews.text"].dropna().tolist()

# STEP 3: Load ATEPC aspect-based sentiment model
aspect_extractor = ATEPC.AspectExtractor(
    checkpoint="english",  # You can also try "english"
    auto_device=True  # uses GPU if available
)

# STEP 4: Extract aspects and sentiment
# extracted = aspect_extractor.extract_aspect(
#     inference_source=product_reviews,  # Add the missing inference_source parameter
#     examples=product_reviews,
#     pred_sentiment=True,
#     print_result=False
# )

# # STEP 5: Aggregate aspects by sentiment
# from collections import defaultdict
# aspect_summary = defaultdict(list)

# for entry in extracted:
#     for asp, sent in zip(entry['aspect'], entry['sentiment']):
#         aspect_summary[sent].append(asp)

# # Keep only top 10 unique aspects per sentiment
# def top_aspects(aspect_list):
#     from collections import Counter
#     return [item for item, _ in Counter(aspect_list).most_common(10)]

# positive_aspects = top_aspects(aspect_summary["Positive"])
# negative_aspects = top_aspects(aspect_summary["Negative"])

# # STEP 6: Use LangChain + Groq to give suggestions
# os.environ["GROQ_API_KEY"] = "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"  # Replace with your Groq key

# llm = ChatGroq(model="llama3-70b-8192")

# prompt = PromptTemplate(
#     input_variables=["positives", "negatives"],
#     template="""
# You are a product advisor. Based on these positive and negative aspects from product reviews:

# Positive aspects: {positives}
# Negative aspects: {negatives}

# Give sellers two lists:
# 1. Improvements they should make (based on negative aspects).
# 2. Strengths they should highlight (based on positive aspects).

# Return the output as a Python dictionary with keys 'positive' and 'negative', each having list of strings as suggestions.
# """
# )

# chain =  prompt | llm
# response = chain.invoke({
#     "positives": ", ".join(positive_aspects),
#     "negatives": ", ".join(negative_aspects)
# })

# # Final Output
# print("\n🧠 Suggestion Summary:\n")
# print(response)


[2025-06-21 15:15:32] (2.0.27) ********** Available ATEPC model checkpoints for Version:2.0.27 (this version) **********
[2025-06-21 15:15:32] (2.0.27) ********** Available ATEPC model checkpoints for Version:2.0.27 (this version) **********
[2025-06-21 15:15:32] (2.0.27) Downloading checkpoint:english 
[2025-06-21 15:15:32] (2.0.27) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets


KeyboardInterrupt: 